In [ ]:
import json
import requests
import socket

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import psycopg2
import seaborn as sns
from sqlalchemy import create_engine

In [ ]:
# superset-dbコンテナのjupyterデータベースを指定（変更不可）
engine = create_engine('postgresql://postgres@superset-db:5432/jupyter')

In [ ]:
# ステートメントをAPIサーバから取得（以下はxAPIステートメント取得の例）

## http://docs.learninglocker.net/http-aggregation/
url = 'http://{}:8080/statements/aggregate?cache=false&maxTimeMS=5000&maxScan=10000&pipeline=%5B%7B%22%24limit%22%3A%203000%7D%2C%20%7B%22%24project%22%3A%20%7B%20%22statement%22%3A%201%2C%20%22_id%22%3A%200%20%7D%7D%5D'.format(socket.gethostbyname('learninglocker'))
## Learning Lockerのクライアント情報を設定
user = 'de25a0afae3d21b67a7278a330ce6de115ba3c77'
passwd = '42df984faf8f4ed457be5a85477df23a9e3cc62e'
## Learning LockerのAGGREGATION APIでxAPIステートメントを取得
r = requests.get(url, auth=(user, passwd))
## Jsonを辞書型に変換
response_data = json.loads(r.text)
## 辞書からステートメントのみを抽出
stats = [d['statement'] for d in response_data]


In [ ]:
# ステートメントの分析
# 分析結果は最終的にpandasのDataframe形式とすること

## ネスト項目をフラットにしてpandasのDataFrameを作成
df = pd.io.json.json_normalize(stats)
## 特定ユーザのステートメントを選択
df = df[df['actor.account.name'] == '2']
## オブジェクトとタイムスタンプの射影
df = df[['object.definition.description.en', 'object.id', 'timestamp']]
df

In [ ]:
# Supersetのjupyterデータベースに処理結果を保存
# 次にエラーが表示される場合はテーブル名称を変更すること : ValueError: Table '<tablename>' already exists.

tablename = 'user_2_objects'
df.to_sql(tablename, engine)